In [ ]:
# for numerical work
import pandas as pd
import numpy as np

import pymongo

import datetime
import json

from pandas.io.json import json_normalize
from pymongo import MongoClient

import pickle

# load the database credentials from file
with open('../creds/creds.json') as json_data:
    creds = json.load(json_data)
    
client = MongoClient(creds['connection_string'])

In [ ]:
subset = pd.read_hdf('../lstm_data_prep_pipeline/results/all_request_events.hdf5', 'table')

In [ ]:
subset.head()

In [ ]:
ids = subset.request_id.unique()
ints = np.arange(0, len(ids))

request_id_mapper = dict(zip(ids, ints))

# get the dimensions of the data
n_examples = subset.request_id.unique().size
n_timesteps = subset.timesteps.max()+1 # because the indexing starts at 0 so the event with index 10 is actually the 11th event
n_features = subset.columns.drop(['request_id','index_int','timesteps']).size
features = list(subset.columns.drop(['request_id','index_int','timesteps']))

# create a boolean map to set the na_columns to 1
na_col_map = np.array([i for i, col in enumerate(features) if '_NaN' in col])

# set up the empty dataframe
data = np.zeros((n_examples,n_timesteps,n_features))

request_ids = subset.request_id.values
timestep_ids = subset.timesteps.values

# initialize all the na cols to 1                      
# data[:,:,na_col_map] = 1.0

# get only the features
subset.drop(['request_id','index_int','timesteps'], axis=1, inplace=True)

In [ ]:
import sys

sys.getsizeof(subset)/1000/1000/1000

In [ ]:
subset = subset.values

In [ ]:
# for each row in the dataframe of request events
for i in range(subset.shape[0]):
    
    # print out a status every 1000 records
    if (i+1)%1000 == 0:
        print(i, 'events processed out of', subset.shape[0])
    # get the example index
    example_id = int(request_id_mapper[request_ids[i]])
    
    # get the timestep index
    timestep_id = int(timestep_ids[i])
    
    # get the values index
    values = subset[i,:].values.astype('float64')
    
    # update the values of that example
    data[example_id, timestep_id,:] = values

print("Shape of dataset:",data.shape)



# set up data
X = data
y = fraud
groups = np.array(user_emails)

# create a dict with the results
results_dict = {'X': X,
                'y': y, 
                'groups': groups, 
                'feature_names': np.array(data_df.columns)}

print("Saving results to a pickle file.")

# save results to a pickle file
with open("../lstm_data_prep_pipeline/results/all_request_results.pickle", 'wb') as outfile:
    pickle.dump(results_dict, outfile, protocol=pickle.HIGHEST_PROTOCOL)